# Objetivo 1

# 1.Creación de datos

Se va a crear la matriz X, que es la que contiene las imágenes (se crea un dataset).

In [19]:
import cv2
import glob
import skimage


#Librerias contornos 
import numpy as np
from skimage.io import imread, imshow
from skimage.filters import prewitt_h,prewitt_v
from skimage import data, io
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.filters import threshold_otsu 
from skimage.feature import hog
from skimage import data, exposure

#Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris #dataset de prueba
from sklearn.preprocessing import OneHotEncoder
from scipy.optimize import minimize
import sklearn.metrics as metrics



$devolverDiccionarioEtiquetas$

In [2]:
# $Sift$
#devuelve descriptor de Sift
def siftMaker(images00000_mod):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(images00000_mod,None)    
    #quizas falta un tercer parametro:  imagen original
    img=cv2.drawKeypoints(images00000_mod,kp, images00000_mod, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    #cv2.imwrite('sift_keypoints.jpg',img)
    return des


# $Surf$
def surfmaker(images00000_mod):
    surf = cv2.xfeatures2d.SURF_create(400)
    kp, des = surf.detectAndCompute(images00000_mod,None)
    return des

# $Fast$ + $Brief$
def fastbrief_Maker(images00000_mod):
    fast = cv2.FastFeatureDetector_create()

    # find and draw the keypoints
    kp2 = fast.detect(images00000_mod,None)
    img2 = cv2.drawKeypoints(images00000_mod, kp2, None, color=(255,0,0))

#     # Print all default params
#     print( "Threshold: {}".format(fast.getThreshold()) )
#     print( "nonmaxSuppression:{}".format(fast.getNonmaxSuppression()) )
#     print( "neighborhood: {}".format(fast.getType()) )
#     print( "Total Keypoints with nonmaxSuppression: {}".format(len(kp2)) )

    # cv2.imwrite('fast_true.png',img2)
#     cv2.imshow("test1", img2)

    # Disable nonmaxSuppression
    fast.setNonmaxSuppression(0)
    kp3 = fast.detect(img2,None)

#     print( "Total Keypoints without nonmaxSuppression: {}".format(len(kp3)) )

    img3 = cv2.drawKeypoints(img2, kp3, None, color=(255,0,0))
    #cv2.imshow("test", img3)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    # cv2.imwrite('fast_false.png',img3)
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

    # Compute descriptors
    kp, des = brief.compute(img3, kp3) #fast nonsup
    return des


# $ORB$
def orbMaker(images00000_mod):
    # Initiate ORB detector
    orb = cv2.ORB_create()

    # find the keypoints with ORB
    kp = orb.detect(images00000_mod,None)

    # compute the descriptors with ORB
    kp, des = orb.compute(images00000_mod, kp)

    # draw only keypoints location,not size and orientation
    img2 = cv2.drawKeypoints(images00000_mod, kp, None, color=(0,255,0), flags=0)
    #plt.imshow(img2), plt.show()
    return des

In [3]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son el tipo de signals que contiene.
'''


def devolverDiccionarioEtiquetas():
    
    diccionarioValores = {}
    diccionarioValores['00000'] = 'VelocidadMaxima20'
    diccionarioValores['00001'] = 'VelocidadMaxima30'
    diccionarioValores['00002'] = 'VelocidadMaxima50'
    diccionarioValores['00003'] = 'VelocidadMaxima60'
    diccionarioValores['00004'] = 'VelocidadMaxima70'
    diccionarioValores['00005'] = 'VelocidadMaxima80'
    diccionarioValores['00006'] = 'FinVelocidadMaxima80'
    diccionarioValores['00007'] = 'VelocidadMaxima100'
    diccionarioValores['00008'] = 'VelocidadMaxima120'
    diccionarioValores['00009'] = 'AdelantamientoProhibido'
    diccionarioValores['00010'] = 'AdelantamientoProhibidoParaCamiones'

    diccionarioValores['00011'] = 'InterseccionConPrioridad'
    diccionarioValores['00012'] = 'CalzadaConPrioridad'
    diccionarioValores['00013'] = 'CedaElPaso'
    diccionarioValores['00014'] = 'DetencionObligatoria'
    diccionarioValores['00015'] = 'CirculacionProhibida'
    diccionarioValores['00016'] = 'EntradaProhibidaAVehiculosMercancias'
    diccionarioValores['00017'] = 'EntradaProhibida'
    diccionarioValores['00018'] = 'OtrosPeligros'
    diccionarioValores['00019'] = 'CurvaPeligrosaHaciaLaIzquierda'
    diccionarioValores['00020'] = 'CurvaPeligrosaHaciaLaDerecha'

    diccionarioValores['00021'] = 'CurvasPeligrosasHaciaLaIzquierda'
    diccionarioValores['00022'] = 'PerfilIrregular'
    diccionarioValores['00023'] = 'PavimentoDeslizante'
    diccionarioValores['00024'] = 'EstrechamientoCalzadaPorDerecha'
    diccionarioValores['00025'] = 'Obras'
    diccionarioValores['00026'] = 'Semaforos'
    diccionarioValores['00027'] = 'PasoDePeatones'
    diccionarioValores['00028'] = 'Kids'
    diccionarioValores['00029'] = 'EntradaProhibidasCiclos'
    diccionarioValores['00030'] = 'PavimentoDeslizanteNieveHielo'

    diccionarioValores['00031'] = 'PasoDeAnimalesEnLibertad'
    diccionarioValores['00032'] = 'FinDeProhibiciones'
    diccionarioValores['00033'] = 'SentidoObligatorioDerecha'
    diccionarioValores['00034'] = 'SentidoObligatorioIzquierda'
    diccionarioValores['00035'] = 'SentidoObligatorioArriba'
    diccionarioValores['00036'] = 'DirPermitidasArribaDerecha'
    diccionarioValores['00037'] = 'DirPermitidasArribaIzquierda'
    diccionarioValores['00038'] = 'PasoObligatorioDerecha'
    diccionarioValores['00039'] = 'PasoObligatorioizquierda'
    diccionarioValores['00040'] = 'InterseccionSentidoObligatorioGiratorio'


    diccionarioValores['00041'] = 'FinProhibicionAdelantamiento'
    diccionarioValores['00042'] = 'FinProhibicionAdelantamientoCamiones'
    
    return diccionarioValores

Probemos ahora el código....

In [4]:
d =  devolverDiccionarioEtiquetas()

for k,v in d.items():
    print(v)

VelocidadMaxima20
VelocidadMaxima30
VelocidadMaxima50
VelocidadMaxima60
VelocidadMaxima70
VelocidadMaxima80
FinVelocidadMaxima80
VelocidadMaxima100
VelocidadMaxima120
AdelantamientoProhibido
AdelantamientoProhibidoParaCamiones
InterseccionConPrioridad
CalzadaConPrioridad
CedaElPaso
DetencionObligatoria
CirculacionProhibida
EntradaProhibidaAVehiculosMercancias
EntradaProhibida
OtrosPeligros
CurvaPeligrosaHaciaLaIzquierda
CurvaPeligrosaHaciaLaDerecha
CurvasPeligrosasHaciaLaIzquierda
PerfilIrregular
PavimentoDeslizante
EstrechamientoCalzadaPorDerecha
Obras
Semaforos
PasoDePeatones
Kids
EntradaProhibidasCiclos
PavimentoDeslizanteNieveHielo
PasoDeAnimalesEnLibertad
FinDeProhibiciones
SentidoObligatorioDerecha
SentidoObligatorioIzquierda
SentidoObligatorioArriba
DirPermitidasArribaDerecha
DirPermitidasArribaIzquierda
PasoObligatorioDerecha
PasoObligatorioizquierda
InterseccionSentidoObligatorioGiratorio
FinProhibicionAdelantamiento
FinProhibicionAdelantamientoCamiones


In [5]:
def crearDiccionarioDatos():
    trafic_signals = {}
    #para cada tipo de señal creamos una lista con sus ejemplos, y se la asignamos a su etiqueta de nombre de carpeta
    for i in range(0,43):
        if i<10:
            folder = 'Dataset_traffic_sign/0000'+str(i)+'/*.png'
            tipo = '0000'+str(i)
        else:
            folder = 'Dataset_traffic_sign/000'+str(i)+'/*.png'
            tipo = '000'+str(i)
        
        images = [cv2.imread(file,0) for file in glob.glob(folder)]
        images_mod = [] 
        for im in images:
            #se aplica el resize
            images_mod.append(cv2.resize(im, (100, 100), interpolation = cv2.INTER_NEAREST))

        trafic_signals[tipo] = images_mod
    return trafic_signals

# Creamos nuestra matriz de datos
def creaMatrizCaracteristicas(trafic_signals, signals_types, carac_type):
    X = 0
    Y = 0
    
    for signal,images in trafic_signals.items():
        y = []
        x = []
        
        x = obtenerDesCaracteristicas(np.array(images), carac_type)
        
        for i in range(x.shape[0]):
            y.append(signals_types[signal])
            
        y = np.array(y).reshape(-1,1)
        
        if signal=='00000':
            Y = y
            X = x
        else:
            X = np.vstack((X,x))
            Y = np.vstack((Y,y))
    
    return X,Y

# Pasamos una np array de imagenes y vamos sacando sus descriptores 1 a 1
def obtenerDesCaracteristicas(images_o, carac_type):
    des = None
    
    for i in range(images_o.shape[0]):
        if carac_type=='sift':
            d = siftMaker(images_o[i])
        elif carac_type=='surf':
            d = surfmaker(images_o[i])
        elif carac_type=='fast_brief':
            d = fastbrief_Maker(images_o[i])
        elif carac_type=='orb':
            d = orbMaker(images_o[i])
        else:
            try:
                raise ValueError
            except ValueError:
                print("valor incorrecto")
                
        if d is not None:
            if des is None:
                des = d
            else:
                des = np.vstack((des,d))
    return des

In [10]:
trafic_signals = crearDiccionarioDatos()
signals_types = devolverDiccionarioEtiquetas()
carac_type = 'orb'

In [11]:
X, y = creaMatrizCaracteristicas(trafic_signals, signals_types, carac_type)

In [12]:
X.shape, y.shape

((22883, 32), (22883, 1))

In [13]:
X[0]


array([ 41, 162, 228,  55, 116,  30, 120,  48, 119, 237, 119, 205, 107,
        40, 150, 217,  30, 236, 229,  13, 172, 248,  77, 140,  78, 127,
       112, 129, 111, 142,  90,  93], dtype=uint8)

# 2.Aplicación de Machine Learning

A continuación se implementarán dos modelos de ML como son regresión logísitica y redes neuronales para dar solución al problema.

# Regresión logística

Nos hemos inclinado por RL en detrimento de SVM pues el número de características es bastante superior al número de ejemplos...

In [1]:
def transformarY():
    print("Y falta por transformar")

In [2]:
diccionario  = devolverDiccionarioEtiquetas()

NameError: name 'devolverDiccionarioEtiquetas' is not defined

In [3]:
'''
IDEA POR ENCIMA....
#Y = transformarY(Y)
for k,v in diccionario.items():
    Y = nuevaY
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)
prediccion = clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
score = clf.score(X, y)
print(f"La precision del clasificador es : {score}")
print(f"La prediccion es : {prediccion}")
'''


'\nIDEA POR ENCIMA....\n#Y = transformarY(Y)\nfor k,v in diccionario.items():\n    Y = nuevaY\nX, y = load_iris(return_X_y=True)\nclf = LogisticRegression(random_state=0).fit(X, y)\nprediccion = clf.predict(X[:2, :])\nclf.predict_proba(X[:2, :])\nscore = clf.score(X, y)\nprint(f"La precision del clasificador es : {score}")\nprint(f"La prediccion es : {prediccion}")\n'

In [5]:
X, y = load_iris(return_X_y=True)
clf = LogisticRegression(random_state=0).fit(X, y)
prediccion = clf.predict(X[:2, :])
clf.predict_proba(X[:2, :])
score = clf.score(X, y)
print(f"La precision del clasificador es : {score}")
print(f"La prediccion es : {prediccion}")

C:\Users\alemt\anaconda3\envs\py37vision\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\alemt\anaconda3\envs\py37vision\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


La precision del clasificador es : 0.96
La prediccion es : [0 0]


# Neural Networks 

In [ ]:

encoder = OneHotEncoder(sparse=False)
# Aplica el fit_transform de scikit a y para obtener el nuevo y_onehot
y_onehot = encoder.fit_transform(y.reshape(-1,1))

#esto quiere decir que hay 'n' posibles clases. Se pondra un 1 en la clase que participa, y 0 en la clase que no participa.
#La participacion va por  'registros'
#Para mas info: https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f
y_onehot

In [7]:
def sigmoid(z):
    return (1/(1+np.exp(-z)))

In [8]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    bias = np.ones(m).reshape(-1,1)
    
    # Añadimos la columna de unos a X para obtener a1
    a1 = np.hstack((bias,X))
    
    # Calculamos z2 -> ten en cuenta que a1 tiene tantas filas como ejemplos y columnas como atributos + 1
    # Por otro lado, theta1 tiene tantas filas como neuronas en la capa oculta y columnas como atributos + 1
    z2 = a1*theta1.T
    
    # Añadimos la columna de unos a la sigmoide de z2 (que es a2) para obtener el a2 definitivo
    a2 = np.hstack((np.ones(z2.shape[0]).reshape(-1,1),sigmoid(z2)))
    
    # Calculamos z3
    z3 = a2*theta2.T
    
    # Obtenemos la salida final en h
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [9]:
def cost(params, input_size, hidden_size, num_labels, X, y):
    m = X.shape[0]
    # Utilizamos las matrices de numpy por facilidad: * es el producto de matrices y np.multiply elemento por elemento
    X = np.matrix(X)
    y = np.matrix(y)
    
    # desempaquetamos las matrices con los parámetros para cada capa
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # Ejecutamos las propagación hacia adelante para obtener las salidas para cada ejemplo
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # Calculamos el coste
    J = (1/m)*(np.sum(np.multiply((-y),np.log(h))-np.multiply((1-y),np.log(1-h))))
    
    return J

In [10]:
# Configuración inicial
input_size = X.shape[1] #numero de caracteristicas de X
hidden_size = 25
num_labels = 3
np.random.seed(123456789)

# Inicializamos los parámetros de la red aleatoriamente
# El tamaño del array es el tamaño de las dos matrices de pesos concatenadas
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

# Podemos desempaquetar los parámetros que acabamos de inicializar igual que lo hacemos en la función de coste
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

# Veamos si los tamaños de las matrices theta1 y theta2 son correctos
theta1.shape, theta2.shape

((25, 5), (3, 26))

In [11]:
lambda_reg = 1.0 #Parametro regularizador

In [12]:
def costReg(params, input_size, hidden_size, num_labels, X, y, lambda_reg):
    m = X.shape[0]
    # Utilizamos las matrices de numpy por facilidad: * es el producto de matrices y np.multiply elemento por elemento
    X = np.matrix(X)
    y = np.matrix(y)
    
    # desempaquetamos las matrices con los parámetros para cada capa, obtener theta1 y theta2
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # Ejecutamos las propagación hacia adelante para obtener las salidas para cada ejemplo
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # Calculamos el coste
    J = (1/m)*(np.sum(np.multiply((-y),np.log(h))-np.multiply((1-y),np.log(1-h))))
    
    # Es decir, la suma de los parámetros al cuadrado sin considerar la primera columna en ninguna de las dos matrices de parámetros
    J += (lambda_reg/(2*m))*(np.sum(np.sum(np.multiply(theta1,theta1))+np.sum(np.multiply(theta2,theta2))))
    
    return J

In [13]:
def sigmoid_gradient(z):
    g = sigmoid(z)
    return np.multiply(g,(1-g))

In [14]:
def backpropReg(params, input_size, hidden_size, num_labels, X, y, lambda_reg):
    ###################################################################
    # Copiar aquí el código de la función de coste con regularización #
    ###################################################################
    
    m = X.shape[0]
    # Utilizamos las matrices de numpy por facilidad: * es el producto de matrices y np.multiply elemento por elemento
    X = np.matrix(X)
    y = np.matrix(y)
    
    # desempaquetamos las matrices con los parámetros para cada capa, obtener theta1 y theta2
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # Ejecutamos las propagación hacia adelante para obtener las salidas para cada ejemplo
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # Calculamos el coste
    J = (1/m)*(np.sum(np.multiply((-y),np.log(h))-np.multiply((1-y),np.log(1-h))))
    
    # Es decir, la suma de los parámetros al cuadrado sin considerar la primera columna en ninguna de las dos matrices de parámetros
    J += (lambda_reg/(2*m))*(np.sum(np.sum(np.multiply(theta1,theta1))+np.sum(np.multiply(theta2,theta2))))
    
     ############################
    # Comienza Backpropagation #
    ############################
    # Inicializamos los acumuladores delta1  y delta2 a ceros, con las dismensiones de los theta1 y theta2
    # tendrán dimensiones (25, 401) y (10, 26), respectivamente
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    
    # Aunque podríamos vectorizarlo vamos a hacerlo para cada ejemplo
    for t in range(m):
        # Obtenemos lo que necesitamos del ejemplo (cálculos obtenidos en la propagación hacia adelante)
        # Para usar las fórmulas tal y como aparecen, vamos a coger todos los vectores en forma columna (resahpe(-1,1))
        a1t = a1[t,:].reshape(-1, 1)  # (401, 1)
        z2t = z2[t,:].reshape(-1, 1)  # (25, 1)
        a2t = a2[t,:].reshape(-1, 1)  # (26, 1)
        ht = h[t,:].reshape(-1, 1)  # (10, 1)
        yt = y[t,:].reshape(-1, 1)  # (10, 1)
        
        # Calculamos el error en la capa de salida (delta3), almacenar en d3t
        d3t = ht - yt
        
        # Para calcular el error en la capa oculta (delta2) necesitamos añadir un uno al inicio del vector z2t
        # Almacenar en z2t
        z2t = np.vstack((1,z2t))
        
        # Calculamos d2 a partir del error de la capa de salida, los parámetros en theta2 y el gradiente de z2t (guardar en d2t)
        # <RELLENAR>
        d2t = np.multiply(theta2.T*d3t,sigmoid_gradient(z2t))
        
        # Ya podemos calcular los gradientes a partir de los errores
        # Para calcular el gradiente de los theta1, tenemos en cuenta el error en la capa oculta d2
        # Acumular el gradiente del ejemplo en delta1 y delta2
        delta1 += d2t[1:]*a1t.T
        delta2 += d3t*a2t.T
    
    # Calculamos el gradiente finalmente dividiendo entre el número de ejemplos
    delta1 = delta1/m
    delta2 = delta2/m
    
    # Añadimos el término de regularización en delta1 y delta2 (no regularizar el bias)
    
    delta1[:,1:] += lambda_reg*theta1[:,1:]/m
    delta2[:,1:] += lambda_reg*theta2[:,1:]/m
    
    # Para pasar los gradientes a minimize los ponemos en un vector
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [15]:
J, grad = backpropReg(params, input_size, hidden_size, num_labels, X, y_onehot, lambda_reg)
print(J, grad.shape)

1.9785097686071078 (203,)


# Entrenamiento de la red y clasificación

Ahora ya estamos listos para entrenar la red y usarla para hacer predicciones. Para entrenarla, utilizamos el método `minimize`de scipy, indicándole que backpropReg es la función que calcula el coste y los gradientes.


In [16]:
from scipy.optimize import minimize

# Minimizar la función objetivo que acabamos de definir
fmin = minimize(fun=backpropReg, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, lambda_reg), 
                method='TNC', jac=True, options={'maxiter': 250})

In [17]:
# Desempaquetamos los parámetros obtenidos como resultado del entrenamiento almacenados en fmin.x
# Crea las variables theta1 y theta2
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

# Utilizamos los parámetros desempaquetados con la propagación hacia adelante para obtener la predicción para nuestros ejemplos
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

# Finalmente, para obtener la clase para cada ejemplo, buscamos de las diez salidas cual es la más alta
# y usamos su índice como valor predicho (utilizar np.argmax con axis=1 que hace precísamente eso).
y_pred = np.argmax(h,axis=1)

In [20]:
accTrain = metrics.accuracy_score(y_pred, y)

print("Precisión en train: {}%".format(accTrain*100))

Precisión en train: 98.66666666666667%
